<a href="https://colab.research.google.com/github/mahesh-tippanu/liberay_bPML/blob/b-feature%2Fdata-preprocessing/liberay_bPML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preprocessing

In [1]:

import pandas as pd

file_path = '/content/drive/MyDrive/Machine Learning Project/Ist Quratile journals Scopus (1).xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Handle missing values
df['SNIP'].fillna(df['SNIP'].median(), inplace=True)
df['SJR'].fillna(df['SJR'].median(), inplace=True)
df['Publisher'].fillna(df['Publisher'].mode()[0], inplace=True)
df['Main Publisher'].fillna(df['Main Publisher'].mode()[0], inplace=True)

# Drop columns with excessive missing values
df_cleaned = df.drop(columns=['Print ISSN', 'E-ISSN'])

# Handle duplicate entries
df_cleaned.drop_duplicates(inplace=True)

# Save the cleaned data for the next step
df_cleaned.to_excel('cleaned_journal_data.xlsx', index=False)


<ipython-input-1-8a314bc0e57a>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SNIP'].fillna(df['SNIP'].median(), inplace=True)
<ipython-input-1-8a314bc0e57a>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us